# VelMANNAutoregressive
This notebook contains the code showing how to use VelMANNAutoregressive to generate a walking trajectory for the ergoCub humanoid robot. 

If you are interested in some details of the algorithm, please refer to the paper: [Trajectory Generation with Physics-Informed Learning and Drift Mitigation](https://github.com/ami-iit/paper_delia_2024_ral_physics-informed_trajectory_generation).

## Import all the required packages
In this section we import all the required packages. In particular, we need `iDynTree` to correctly visualize the robot, `numpy` to perform some basic operations, `manifpy` to perform some basic operations on manifolds, `resolve_robotics_uri_py` to correctly locate the `ergoCub` model. Finally `bipedal_locomotion_framework` implements the MANN network.

In [1]:
from idyntree.visualize import MeshcatVisualizer
import idyntree.bindings as idyn
from pathlib import Path
import numpy as np
import manifpy as manif
import bipedal_locomotion_framework as blf
import resolve_robotics_uri_py

## Prepare the ingredients
To correctly run the notebook, we need to prepare the ingredients. In particular, we need to:
- Get the network model: In this case we use the model trained on the ergoCub robot. The velocity-based model is available [here](https://huggingface.co/ami-iit/mann/resolve/main/ergocubSN000_26j_49e_velbased_mann.onnx).
- Load the configuration file: The configuration file contains all the parameters needed to correctly run the network and generate a proper set of inputs for the network. In detail we have two configuration files:
    - `config_mann.toml`: This file contains the parameters needed to correctly load and run the network.
    - `config_joypad.toml`: This file contains the parameters needed to correctly generate the input for the network. In particular, it contains the parameters needed to correctly map a joypad input to the network input.
- Load the robot model: In this case we use the `ergoCub` model. We load the model only to correctly visualize the robot.

### Get the network model
What we need to do is to download the network model and save it in the `config` folder. The model is available [here](https://huggingface.co/ami-iit/mann/resolve/main/ergocubSN000_26j_49e_velbased_mann.onnx). 
Moreover we load the parameters needed to correctly run the network.

In [ ]:
# We use urllib to download the onnx model from huggingface
import urllib.request
import os
url = "https://huggingface.co/datasets/evelyd/paper_delia_2024_ral_physics-informed_trajectory_generation_dataset/resolve/main/onnx/training_subsampled_mirrored_10x_pi_20240514-173315_ep130.onnx"
urllib.request.urlretrieve(url, "./config/training_subsampled_mirrored_10x_pi_20240514-173315_ep130.onnx")

# Get the configuration files
config_path = Path("__file__").parent / "config" / "config_mann.toml"
params_network = blf.parameters_handler.TomlParametersHandler()
params_network.set_from_file(str(config_path))

joypad_config_path = Path("__file__").parent / "config" / "config_joypad.toml"
params_joypad = blf.parameters_handler.TomlParametersHandler()
params_joypad.set_from_file(str(joypad_config_path))

### Load ergoCub model
We load the `ergoCub` model to correctly visualize the robot.
The model is loaded using `iDynTree` and `resolve_robotics_uri_py` to correctly locate the model. Please notice that the model is loaded specifying the same joint order used to train the network.

In [ ]:
# Get the path of the robot model
robot_model_path = str(resolve_robotics_uri_py.resolve_robotics_uri("package://ergoCub/robots/ergoCubSN001/model.urdf"))
ml = idyn.ModelLoader()
ml.loadReducedModelFromFile(robot_model_path, params_network.get_parameter_vector_string("joints_list"))
viz = MeshcatVisualizer()
viz.load_model(ml.model())

## Network initialization
In this section we initialize the network. In particular we first instantiate the network and then we load the parameters. The parameters are loaded from the `config_mann.toml` file.

In [ ]:
# Create the trajectory generator
mann_trajectory_generator = blf.ml.VelMANNAutoregressive()
assert mann_trajectory_generator.set_robot_model(ml.model())
assert mann_trajectory_generator.initialize(params_network)

# Create the input builder
input_builder = blf.ml.VelMANNAutoregressiveInputBuilder()
assert input_builder.initialize(params_joypad)

We now need to reset the network to the initial state. In particular, we need to reset  the joint state and the base pose in a given configuration already seen by the network during traning.

In [5]:
# Initial joint positions configuration. The serialization is specified in the config file
joint_positions = params_network.get_parameter_vector_float("initial_joints_configuration")

# Initial base pose. This pose makes the robot stand on the ground with the feet flat
base_pose = manif.SE3.Identity()
initial_base_height = params_network.get_parameter_float("initial_base_height")
quat = params_network.get_parameter_vector_float("initial_base_quaternion")
quat = quat / np.linalg.norm(quat) # Normalize the quaternion
base_pose = manif.SE3([0, 0, initial_base_height], quat)

We finally ask the input builder to generate an input that mimics a joypad input. In particular, we ask the input builder to generate an input that mimics a joypad input with the axes moved to the following values:
- `left_stick_x`: 1.0
- `left_stick_y`: 0.0
- `right_stick_x`: 1.0
- `right_stick_y`: 0.0

In [6]:
input_builder_input = blf.ml.VelMANNDirectionalInput()
input_builder_input.motion_direction = np.array([1, 0])
input_builder_input.base_direction = np.array([1, 0])

# second_input_builder_input = blf.ml.VelMANNDirectionalInput()
# second_input_builder_input.motion_direction = np.array([1, 0])
# second_input_builder_input.base_direction = np.array([1, 0])

# third_input_builder_input = blf.ml.VelMANNDirectionalInput()
# third_input_builder_input.motion_direction = np.array([0, 0])
# third_input_builder_input.base_direction = np.array([1, 0])

## Network execution
We now visualize the robot in the viewer and we run the network for 150 steps. In particular, we run the network for 150 steps and we visualize the resulting robot motion at each step.

In [ ]:
viz.jupyter_cell()

In [8]:
# Reset the trajectory generator
mann_trajectory_generator.reset(joint_positions, base_pose)
length_of_time = params_network.get_parameter_int("trajectory_length")
base_translations = np.zeros(shape=(3,length_of_time))
base_rotations = np.zeros(shape=(3,length_of_time))
foot_contacts = np.zeros(shape=(2,length_of_time))
left_foot_velocities = np.zeros(shape=(6,length_of_time))
right_foot_velocities = np.zeros(shape=(6,length_of_time))
left_foot_positions = np.zeros(shape=(3,length_of_time))
left_foot_rotations = np.zeros(shape=(3,length_of_time))
right_foot_positions = np.zeros(shape=(3,length_of_time))
right_foot_rotations = np.zeros(shape=(3,length_of_time))

from scipy.spatial.transform import Rotation as R

for i in range(length_of_time):

    # Set the input to the builder and
    # if i < (length_of_time/4):
    input_builder.set_input(input_builder_input)
    # elif i < (2*length_of_time/4):
        # input_builder.set_input(second_input_builder_input)
    # else:
        # input_builder.set_input(third_input_builder_input)
    assert input_builder.advance()
    assert input_builder.is_output_valid()

    # Set the input to the trajectory generator
    mann_trajectory_generator.set_input(input_builder.get_output())
    assert mann_trajectory_generator.advance()
    assert mann_trajectory_generator.is_output_valid()

    # Get the output of the trajectory generator and update the visualization
    mann_output = mann_trajectory_generator.get_output()
    base_translations[:,i] = np.array(mann_output.base_pose.translation())
    base_rotations[:,i] = R.from_matrix(mann_output.base_pose.rotation()).as_euler('xyz')
    foot_contacts[:,i] = np.array([mann_output.left_foot.is_active, mann_output.right_foot.is_active])
    left_foot_velocities[:,i] = np.array(mann_output.left_foot_velocity)
    right_foot_velocities[:,i] = np.array(mann_output.right_foot_velocity)
    left_foot_positions[:,i] = np.array(mann_output.left_foot_pose.translation())
    right_foot_positions[:,i] = np.array(mann_output.right_foot_pose.translation())
    left_foot_rotations[:,i] = R.from_matrix(mann_output.left_foot_pose.rotation()).as_euler('xyz')
    right_foot_rotations[:,i] = R.from_matrix(mann_output.right_foot_pose.rotation()).as_euler('xyz')

    viz.set_multibody_system_state(mann_output.base_pose.translation(),
                                   mann_output.base_pose.rotation(),
                                   mann_output.joint_positions)

In [ ]:
print(np.sqrt(np.mean(left_foot_velocities[:3,i]**2)))

In [ ]:
# get the mse of the foot vels only when that foot is in contact
left_foot_vel_error = np.zeros(shape=(length_of_time,1))
right_foot_vel_error = np.zeros(shape=(length_of_time,1))
left_foot_ang_vel_error = np.zeros(shape=(length_of_time,1))
right_foot_ang_vel_error = np.zeros(shape=(length_of_time,1))
for i in range(length_of_time):
    if foot_contacts[0,i] > 0.5:
        # then the left foot is in contact, take the root mean squared
        left_foot_vel_error[i] = np.sqrt(np.mean(left_foot_velocities[:3,i]**2))
        left_foot_ang_vel_error[i] = np.sqrt(np.mean(left_foot_velocities[3:,i]**2))
    if foot_contacts[1,i] > 0.5:
        # then the right foot is in contact, take the root mean squared
        right_foot_vel_error[i] = np.sqrt(np.mean(right_foot_velocities[:3,i]**2))
        right_foot_ang_vel_error[i] = np.sqrt(np.mean(right_foot_velocities[3:,i]**2))

print("Total RMSE linear vel left: ", np.sum(left_foot_vel_error))
print("Total RMSE linear vel right: ", np.sum(right_foot_vel_error))
print("Total RMSE angular vel left: ", np.sum(left_foot_ang_vel_error))
print("Total RMSE angular vel right: ", np.sum(right_foot_ang_vel_error))

In [ ]:
import matplotlib as mpl
mpl.rcParams['toolbar'] = 'None'
import matplotlib.pyplot as plt

plt.figure(1)
plt.plot(base_translations[0,:] - base_translations[0,0])
plt.plot(base_translations[1,:] - base_translations[1,0])
plt.plot(base_translations[2,:] - base_translations[2,0])
plt.title("Base translations")
plt.xlabel("Timestep")
plt.ylabel("Displacement (m)")
plt.legend(['x','y','z'])
# plt.savefig("base_translations_pi_loss.png")

plt.figure(2)
plt.plot(base_rotations[0,:] - base_rotations[0,0])
plt.plot(base_rotations[1,:] - base_rotations[1,0])
plt.plot(base_rotations[2,:] - base_rotations[2,0])
plt.title("Base rotations")
plt.xlabel("Timestep")
plt.ylabel("Displacement (rad)")
plt.legend(['roll','pitch','yaw'])
# plt.savefig("base_rotations_pi_loss.png")

plt.figure(3)
plt.plot(foot_contacts[0,:])
plt.plot(foot_contacts[1,:])
plt.title("Foot contacts")
plt.xlabel("Timestep")
plt.ylabel("Height (m)")
plt.legend(['left','right'])
# plt.savefig("foot_contacts_pi_loss.png")

fig = plt.figure(7)
ax = fig.add_subplot(111)
ax.fill_between(range(0,len(foot_contacts[0,:])), foot_contacts[0,:], alpha=0.5)
ax.fill_between(range(0,len(foot_contacts[1,:])), foot_contacts[1,:], alpha=0.5)
num_colors = 3
cm = plt.get_cmap('winter')
ax.set_prop_cycle(color=[cm(1.*i/num_colors) for i in range(num_colors)])
for i in range(num_colors):
    ax.plot(left_foot_velocities[i,:])
num_colors = 3
cm = plt.get_cmap('autumn')
ax.set_prop_cycle(color=[cm(1.*i/num_colors) for i in range(num_colors)])
for i in range(num_colors):
    ax.plot(right_foot_velocities[i,:])
plt.title("Foot velocities")
plt.xlabel("Timestep")
plt.ylabel("Velocity (m/s)")
plt.ylim([-0.1, 0.1])
plt.legend(['left contact', 'right contact', 'left x', 'left y', 'left z', 'right x', 'right y', 'right z'], bbox_to_anchor=(1.05, 1.0), loc='upper left')
# plt.savefig("foot_velocities_pi_loss.png")

fig = plt.figure(8)
ax = fig.add_subplot(111)
ax.fill_between(range(0,len(foot_contacts[0,:])), foot_contacts[0,:], alpha=0.5)
ax.fill_between(range(0,len(foot_contacts[1,:])), foot_contacts[1,:], alpha=0.5)
num_colors = 3
cm = plt.get_cmap('winter')
ax.set_prop_cycle(color=[cm(1.*i/num_colors) for i in range(num_colors)])
for i in range(num_colors):
    ax.plot(left_foot_velocities[i+3,:])
num_colors = 3
cm = plt.get_cmap('autumn')
ax.set_prop_cycle(color=[cm(1.*i/num_colors) for i in range(num_colors)])
for i in range(num_colors):
    ax.plot(right_foot_velocities[i+3,:])
plt.title("Foot angular velocities")
plt.xlabel("Timestep")
plt.ylabel("Velocity (rad/s)")
plt.ylim([-0.1, 0.1])
plt.legend(['left contact', 'right contact', 'left x', 'left y', 'left z', 'right x', 'right y', 'right z'], bbox_to_anchor=(1.05, 1.0), loc='upper left')
# plt.savefig("foot_ang_velocities_pi_loss.png")

fig = plt.figure(9)
ax = fig.add_subplot(111)
ax.fill_between(range(0,len(foot_contacts[0,:])), foot_contacts[0,:], alpha=0.5)
ax.fill_between(range(0,len(foot_contacts[1,:])), foot_contacts[1,:], alpha=0.5)
num_colors = 3
cm = plt.get_cmap('winter')
ax.set_prop_cycle(color=[cm(1.*i/num_colors) for i in range(num_colors)])
for i in range(num_colors):
    ax.plot(left_foot_positions[i,:])
num_colors = 3
cm = plt.get_cmap('autumn')
ax.set_prop_cycle(color=[cm(1.*i/num_colors) for i in range(num_colors)])
for i in range(num_colors):
    ax.plot(right_foot_positions[i,:])
plt.title("Foot positions")
plt.xlabel("Timestep")
plt.ylabel("Position (m)")
plt.legend(['left contact', 'right contact', 'left x', 'left y', 'left z', 'right x', 'right y', 'right z'], bbox_to_anchor=(1.05, 1.0), loc='upper left')
# plt.savefig("foot_positions_pi_loss.png")

fig = plt.figure(10)
ax = fig.add_subplot(111)
ax.fill_between(range(0,len(foot_contacts[0,:])), foot_contacts[0,:], alpha=0.5)
ax.fill_between(range(0,len(foot_contacts[1,:])), foot_contacts[1,:], alpha=0.5)
num_colors = 3
cm = plt.get_cmap('winter')
ax.set_prop_cycle(color=[cm(1.*i/num_colors) for i in range(num_colors)])
for i in range(num_colors):
    ax.plot(left_foot_rotations[i,:])
num_colors = 3
cm = plt.get_cmap('autumn')
ax.set_prop_cycle(color=[cm(1.*i/num_colors) for i in range(num_colors)])
for i in range(num_colors):
    ax.plot(right_foot_rotations[i,:])
plt.title("Foot rotations")
plt.xlabel("Timestep")
plt.ylabel("Angle (rad)")
plt.legend(['left contact', 'right contact', 'left x', 'left y', 'left z', 'right x', 'right y', 'right z'], bbox_to_anchor=(1.05, 1.0), loc='upper left')
# plt.savefig("foot_rotations_pi_loss.png")

plt.figure(11)
plt.fill_between(range(0,len(foot_contacts[0,:])), foot_contacts[0,:], alpha=0.5)
plt.fill_between(range(0,len(foot_contacts[1,:])), foot_contacts[1,:], alpha=0.5)
plt.plot(left_foot_vel_error)
plt.plot(right_foot_vel_error)
plt.title("Foot velocity RMSE")
plt.xlabel("Timestep")
plt.ylabel("RMSE (m/s)")
plt.ylim([0, 0.1])
plt.legend(['left contact', 'right contact', 'left RMSE', 'right RMSE'])
# plt.savefig("foot_vel_errors_pi_loss.png")

plt.figure(12)
plt.fill_between(range(0,len(foot_contacts[0,:])), foot_contacts[0,:], alpha=0.5)
plt.fill_between(range(0,len(foot_contacts[1,:])), foot_contacts[1,:], alpha=0.5)
plt.plot(left_foot_ang_vel_error)
plt.plot(right_foot_ang_vel_error)
plt.title("Foot angular velocity RMSE")
plt.xlabel("Timestep")
plt.ylabel("RMSE (rad/s)")
plt.ylim([0, 0.1])
plt.legend(['left contact', 'right contact', 'left RMSE', 'right RMSE'])
# plt.savefig("foot_ang_vel_errors_pi_loss.png")

plt.show()